In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from scipy.sparse import hstack, csr_matrix
import joblib

In [2]:
# Paso 1: Cargar los datos
data = pd.read_csv('chicago_crimes.csv')

In [3]:
# Paso 2: Preprocesamiento de datos
# Eliminar columnas innecesarias o que no sean útiles para la predicción
data = data.drop(columns=['Case Number', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude', 'Location Description'])

In [4]:
# Convertir las fechas a un formato adecuado
data['Date'] = pd.to_datetime(data['Date'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

In [5]:
# Extraer características relevantes de las fechas
data['YEAR'] = data['Date'].dt.year
data['MONTH'] = data['Date'].dt.month
data['DAY'] = data['Date'].dt.day
data['HOUR'] = data['Date'].dt.hour

In [6]:
# Eliminar la columna original de fecha
data = data.drop(columns=['Date'])

In [7]:
# Manejar los valores faltantes en las columnas numéricas
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
imputer = SimpleImputer(strategy='mean')
data[numeric_columns] = imputer.fit_transform(data[numeric_columns])

In [8]:
# Codificar las variables categóricas con OneHotEncoder
categorical_columns = data.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse_output=True)
data_encoded = encoder.fit_transform(data[categorical_columns])
joblib.dump(encoder, 'onehot_encoder.pkl')

['onehot_encoder.pkl']

In [9]:
# Asegurarse de que todas las columnas numéricas sean del tipo float64
numerical_columns = data.drop(columns=categorical_columns).astype('float64')

In [10]:
# Convertir las columnas numéricas a una matriz dispersa
numerical_columns = csr_matrix(numerical_columns.values)

In [11]:
# Combinar las características codificadas con las numéricas
X_combined = hstack((data_encoded, numerical_columns))

In [12]:
# Dividir los datos en características (X) y etiquetas (y)
y = data['Primary Type']
X = X_combined

In [13]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Reducir el tamaño del conjunto de datos para pruebas iniciales (opcional)
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=0.1, random_state=42)

In [15]:
# Paso 3: Entrenamiento del modelo
model = RandomForestClassifier(n_estimators=50, max_depth=10, max_features='sqrt', n_jobs=-1)
model.fit(X_train_sample, y_train_sample)

RandomForestClassifier(max_depth=10, n_estimators=50, n_jobs=-1)

In [16]:
# Paso 4: Evaluación del modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.3026977292937831


In [17]:
# Paso 5: Guardar el modelo entrenado
joblib.dump(model, 'crime_prediction_model.pkl')

['crime_prediction_model.pkl']